In [1]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
from tensorflow import keras

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(input_shape=(28, 28,1), filters=16, kernel_size=5, strides=2, padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.ReLU(),
  tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=2, padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.ReLU(),
  tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=2, padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.ReLU(),
  #tf.keras.layers.Conv2D(filters=10, kernel_size=5, strides=2, padding='same'),
  tf.keras.layers.Conv2D(filters=10, kernel_size=5, strides=4, padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Activation(activation='sigmoid'),
  #tf.keras.layers.ReLU(),
  #tf.keras.layers.Dropout(0.2),
  #tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  #tf.keras.layers.Dense(10),
  tf.keras.layers.Flatten()
])

"""
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28,1)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])
"""
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test, y_test)

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 60000 samples
10000/10000 [==============================] - 2s 236us/sample - loss: 0.5930 - acc: 0.9828


[0.5930470597267151, 0.9828]

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 16)        416       
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 16)        64        
_________________________________________________________________
re_lu (ReLU)                 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 32)          12832     
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 32)          128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          5

In [4]:
# save weights, model architecture & optimizer to an HDF5 format file
model.save('mnist.h5')

In [5]:
# fetch the tensorflow session using the Keras backend
#tf_session = tf.keras.backend.get_session()

# write out tensorflow checkpoint & meta graph
#tf.saved_model.save(model, "amit")

#saver = tf.compat.v1.train.Saver()
#save_path = saver.save(tf_session,tf_ckpt)

In [6]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
​
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
    return frozen_graph

keras.backend.set_learning_phase(0)
loaded_model= keras.models.load_model('./mnist.h5')

# make list of output and input node names
input_names=[out.op.name for out in loaded_model.inputs]
output_names=[out.op.name for out in loaded_model.outputs]
print('input  node is{}'.format(input_names))
print('output node is{}'.format(output_names))

f = open("input_output_node_name.txt", "w+")
f.write('input  node is{}'.format(input_names) + "\n")
f.write('output node is{}'.format(output_names) + "\n")
f.close()

frozen_graph = freeze_session(keras.backend.get_session(), output_names=output_names)

tf.train.write_graph(frozen_graph, "./", "frozen_graph.pb", as_text=False)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
input  node is['conv2d_input_1']
output node is['flatten_1/Reshape']
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 122 variables.
INFO:tensorflow:Converted 122 variables to const ops.


'./frozen_graph.pb'

In [7]:
"""
#saver = tf.train.Saver()
# session is run
sess = tf.Session()
#initialize the variables
sess.run(tf.global_variables_initializer())
# save the variables
saver.save(sess, './simple_model')
# saving the graph using tf.io.write_graph
tf.io.write_graph(sess.graph, './graph', 'graph.pbtxt')
"""

"\n#saver = tf.train.Saver()\n# session is run\nsess = tf.Session()\n#initialize the variables\nsess.run(tf.global_variables_initializer())\n# save the variables\nsaver.save(sess, './simple_model')\n# saving the graph using tf.io.write_graph\ntf.io.write_graph(sess.graph, './graph', 'graph.pbtxt')\n"

In [8]:
"""
import tensorflow as tf
from tensorflow.python.framework import graph_util

saver = tf.train.import_meta_graph('./simple_model.meta')
graph = tf.get_default_graph()
input_graph_def = graph.as_graph_def()
#sess = tf.Session()

session = keras.backend.get_session()
init = tf.global_variables_initializer()
session.run(init)

saver.restore(sess, "./simple_model")

# this will be seen from the file graph.pbtxt
output_node_names="init"
output_graph_def = graph_util.convert_variables_to_constants(
            session,
            input_graph_def,  
            output_node_names.split(",")
            )  


output_graph= "./simple_model .pb"
print("output_graph...", output_graph)

with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(output_graph_def.SerializeToString())

# closing the session
sess.close()
"""

'\nimport tensorflow as tf\nfrom tensorflow.python.framework import graph_util\n\nsaver = tf.train.import_meta_graph(\'./simple_model.meta\')\ngraph = tf.get_default_graph()\ninput_graph_def = graph.as_graph_def()\n#sess = tf.Session()\n\nsession = keras.backend.get_session()\ninit = tf.global_variables_initializer()\nsession.run(init)\n\nsaver.restore(sess, "./simple_model")\n\n# this will be seen from the file graph.pbtxt\noutput_node_names="init"\noutput_graph_def = graph_util.convert_variables_to_constants(\n            session,\n            input_graph_def,  \n            output_node_names.split(",")\n            )  \n\n\noutput_graph= "./simple_model .pb"\nprint("output_graph...", output_graph)\n\nwith tf.gfile.GFile(output_graph, "wb") as f:\n    f.write(output_graph_def.SerializeToString())\n\n# closing the session\nsess.close()\n'

In [9]:
#[node.name for node in model.inputs]

In [10]:
#[node.name for node in model.outputs]